In [17]:
# Amazon

In [18]:
# Import dependencies
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd

# The page of "100 books to read in a lifetime"
home_url = "https://www.amazon.com/b?ie=UTF8&node=8192263011"

response = requests.get(home_url)
soup = bs(response.text, 'html.parser')

book_urls = []
book_titles=[]

cards = soup.find_all("li", class_="a-carousel-card acswidget-carousel__card")
for card in cards:
    # title
    results = card.find_all("a", class_="a-link-normal")
    for result in results:
        title = result.get("title")
        if title:
            title = str(title).split(" (")[0]
            book_titles.append(title)
            
    # url
    results = card.find_all("a", class_="aok-block aok-nowrap aok-hide-text acs_product-image")
    for result in results:
        url = "https://www.amazon.com" + result["href"]
        book_urls.append(url)

In [19]:
# the following are for each book
# for url in book_urls:

In [20]:
book_urls[0]

'https://www.amazon.com/1984-Signet-Classics-George-Orwell/dp/0451524934'

In [21]:
book_titles[0]

'1984'

In [22]:
book_titles

['1984',
 'A Brief History of Time',
 'A Heartbreaking Work of Staggering Genius',
 'A Long Way Gone: Memoirs of a Boy Soldier',
 'The Bad Beginning: Or, Orphans!',
 'A Wrinkle in Time',
 'Selected Stories, 1968-1994',
 "Alice's Adventures in Wonderland & Through the Looking-Glass",
 "All the President's Men",
 "Angela's Ashes: A Memoir",
 "Are You There God? It's Me, Margaret.",
 'Bel Canto',
 'Beloved',
 'Born to Run: A Hidden Tribe, Superathletes, and the Greatest Race the World Has Never Seen',
 'Breath, Eyes, Memory',
 'Catch-22: 50th Anniversary Edition',
 'Charlie and the Chocolate Factory',
 "Charlotte's Web",
 'Cutting for Stone',
 'Daring Greatly: How the Courage to Be Vulnerable Transforms the Way We Live, Love, Parent, and Lead',
 'Diary of a Wimpy Kid, Book 1',
 'Dune',
 'Fahrenheit 451',
 'Fear and Loathing in Las Vegas: A Savage Journey to the Heart of the American Dream',
 'Gone Girl',
 'Goodnight Moon',
 'Great Expectations',
 'Guns, Germs, and Steel: The Fates of Huma

In [23]:
book_urls[0:5]

['https://www.amazon.com/1984-Signet-Classics-George-Orwell/dp/0451524934',
 'https://www.amazon.com/Brief-History-Time-Stephen-Hawking/dp/0553380168',
 'https://www.amazon.com/Heartbreaking-Work-Staggering-Genius/dp/0375725784',
 'https://www.amazon.com/Long-Way-Gone-Memoirs-Soldier/dp/0374531269',
 'https://www.amazon.com/Bad-Beginning-Orphans-Unfortunate-Events/dp/0061146307']

In [24]:
len(book_urls)

100

In [25]:
len(book_titles)

100

In [26]:
book_urls[0]

'https://www.amazon.com/1984-Signet-Classics-George-Orwell/dp/0451524934'

In [28]:
amazon_books = []
count = 1

for url in book_urls:
    # print(url)
    try:
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        summary = soup.find("div", id="reviewSummary")
        all_ratings=[]

        total_review_count = int(summary.find("span", class_="a-size-medium totalReviewCount").text.replace(",",""))
        averge_rating_text = summary.find("span", class_="arp-rating-out-of-text").text
        averge_rating = averge_rating_text.split(" ")[0]

        #star_ratings = summary.find_all("td", class_="a-text-right aok-nowrap")

        for star_rating in star_ratings:
            rating = summary.find("a", class_="a-size-base a-link-normal 1star histogram-review-count")\
            .text.strip("%")[0]
            all_ratings.append(rating)

        #print("all_ratings:", all_ratings)

        # ISBN
        # content = soup.find("div", class_="content")
        # ul = content.find("ul").find_all("li")
        # text_list = []

        # for li in ul:
        #     li_text = li.text
        #     text_list.append(li_text)
        # ISBN = text_list[3][9:]
        #print("ISBN:", ISBN)

        an_amazon_book = {
        "title": book_titles[book_urls.index(url)],
        "url": url,
        "count": total_review_count,
        "average": averge_rating,
        "5_star": all_ratings[0],
        "4_star": all_ratings[1],
        "3_star": all_ratings[2],
        "2_star": all_ratings[3],
        "1_star": all_ratings[4]
        #"ISBN": ISBN
        # ISBN-10 numbers of books
        }

        print(count, book_urls.index(url), an_amazon_book)
        amazon_books.append(an_amazon_book)
    except:
        pass
    
    count +=1

In [ ]:
amazon_books

In [ ]:
len(amazon_books)

In [ ]:
amazon_df = pd.DataFrame(amazon_books)
amazon_df.head()

In [ ]:
amazon_df

In [ ]:
amazon_df.to_csv("amazon.csv", header=None)

In [15]:
# Goodreads

In [16]:
# for each ISBN":"

In [56]:
from splinter import Browser
from bs4 import BeautifulSoup as bs

goodreads_books = []
count = 1

for i in range(len(book_titles)):

    # Get goodreads_id from json of book search api
    goodreads_api_key = "sreSHYqkqprEgfgPBKZw&q="
    title = book_titles[i]
    url = "https://www.goodreads.com/search/index.xml?key=" + goodreads_api_key + title
    response = requests.get(url)
    soup = bs(response.text, 'lxml')
    # use try-except to pass books that are not in goodreads
    try:
        goodreads_id = soup.find("best_book").find("id").text
    except:
        pass

    # to avoid popup login window, I used search url instead of a normal one.
    url = "https://www.goodreads.com/book/show/" + goodreads_id + "." + title + "?from_search=true"
        
    executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
    browser = Browser('chrome', **executable_path, headless=False)
    browser.visit(url)

    try:
        with browser as b:
            # stuff using the browser

            # How do you scrape behind login auth page using Python Requests?
            # close popup window and scrape

            #browser.find_by_css("a#rating_details_tip.gr-hyperlink").first.click()
            browser.find_by_css("a#rating_details.actionLinkLite").first.click()

            html = browser.html
            soup = bs(html, 'html.parser')
            total_review_count = soup.find("span", class_="votes value-title").text.strip()
            averge_rating = int(soup.find("span", class_="average").text.strip())

            result = soup.find("div", class_="content clearfix")
            td_list = result.find_all("td",{"width":"90"})

        #total_review_count, averge_rating

        all_ratings = []
        for td in td_list:
            rating = int(td.text.strip().split("%")[0])
            all_ratings.append(rating)
        all_ratings

        a_goodreads_book = {
        "title": book_titles[i],
        "url": book_urls[i],
        "count": total_review_count,
        "average": averge_rating,
        "5_star": all_ratings[0],
        "4_star": all_ratings[1],
        "3_star": all_ratings[2],
        "2_star": all_ratings[3],
        "1_star": all_ratings[4],
        #"ISBN": ISBN,
        # ISBN-10 numbers of books
        "goodreads_id": goodreads_id
        }

        print(count, i, a_goodreads_book)
        count += 1
        goodreads_books.append(a_goodreads_book)
    except:
        pass

1 0 {'title': '1984', 'url': 'https://www.amazon.com/1984-Signet-Classics-George-Orwell/dp/0451524934', 'count': '2,352,493', 'average': '4.16', '5_star': 45, '4_star': 33, '3_star': 15, '2_star': 4, '1_star': 2, 'goodreads_id': '40961427'}
2 1 {'title': 'A Brief History of Time', 'url': 'https://www.amazon.com/Brief-History-Time-Stephen-Hawking/dp/0553380168', 'count': '212,389', 'average': '4.15', '5_star': 41, '4_star': 38, '3_star': 16, '2_star': 3, '1_star': 1, 'goodreads_id': '3869'}
3 2 {'title': 'A Heartbreaking Work of Staggering Genius', 'url': 'https://www.amazon.com/Heartbreaking-Work-Staggering-Genius/dp/0375725784', 'count': '156,244', 'average': '3.68', '5_star': 27, '4_star': 33, '3_star': 23, '2_star': 10, '1_star': 5, 'goodreads_id': '4953'}
4 3 {'title': 'A Long Way Gone: Memoirs of a Boy Soldier', 'url': 'https://www.amazon.com/Long-Way-Gone-Memoirs-Soldier/dp/0374531269', 'count': '138,691', 'average': '4.15', '5_star': 41, '4_star': 37, '3_star': 16, '2_star': 3, 

33 32 {'title': 'Jimmy Corrigan: The Smartest Kid on Earth', 'url': 'https://www.amazon.com/Jimmy-Corrigan-Smartest-Pantheon-Graphic/dp/0375714545', 'count': '20,106', 'average': '4.09', '5_star': 47, '4_star': 28, '3_star': 14, '2_star': 6, '1_star': 3, 'goodreads_id': '34072'}
34 33 {'title': 'Kitchen Confidential Updated Edition: Adventures in the Culinary Underbelly', 'url': 'https://www.amazon.com/Kitchen-Confidential-Updated-Adventures-Underbelly/dp/0060899220', 'count': '162,997', 'average': '4.02', '5_star': 33, '4_star': 41, '3_star': 20, '2_star': 3, '1_star': 1, 'goodreads_id': '33313'}
35 34 {'title': 'Life After Life: A Novel', 'url': 'https://www.amazon.com/Life-After-Novel-Kate-Atkinson/dp/0316176494', 'count': '271,984', 'average': '3.84', '5_star': 29, '4_star': 36, '3_star': 24, '2_star': 6, '1_star': 2, 'goodreads_id': '28580921'}
36 35 {'title': 'Little House on the Prairie', 'url': 'https://www.amazon.com/Little-House-Prairie-Full-Color/dp/0061958271', 'count': '22

65 64 {'title': 'The Giver', 'url': 'https://www.amazon.com/Giver-Lois-Lowry/dp/0385732554', 'count': '1,420,604', 'average': '4.12', '5_star': 43, '4_star': 33, '3_star': 16, '2_star': 4, '1_star': 1, 'goodreads_id': '3636'}
66 65 {'title': 'The Golden Compass: His Dark Materials', 'url': 'https://www.amazon.com/Golden-Compass-His-Dark-Materials/dp/0440238137', 'count': '1,078,435', 'average': '3.95', '5_star': 38, '4_star': 31, '3_star': 19, '2_star': 6, '1_star': 3, 'goodreads_id': '119322'}
67 66 {'title': 'The Great Gatsby', 'url': 'https://www.amazon.com/Great-Gatsby-F-Scott-Fitzgerald/dp/0743273567', 'count': '3,087,729', 'average': '3.9', '5_star': 34, '4_star': 33, '3_star': 21, '2_star': 6, '1_star': 3, 'goodreads_id': '4671'}
68 67 {'title': "The Handmaid's Tale", 'url': 'https://www.amazon.com/Handmaids-Tale-Margaret-Atwood/dp/038549081X', 'count': '898,950', 'average': '4.08', '5_star': 40, '4_star': 36, '3_star': 16, '2_star': 4, '1_star': 1, 'goodreads_id': '38447'}
69 6

97 97 {'title': 'Valley of the Dolls', 'url': 'https://www.amazon.com/Valley-Dolls-Jacqueline-Susann/dp/0802135196', 'count': '41,482', 'average': '3.72', '5_star': 26, '4_star': 33, '3_star': 28, '2_star': 9, '1_star': 2, 'goodreads_id': '581811'}
98 98 {'title': 'Where the Sidewalk Ends: The Poems and Drawings of Shel Silverstein', 'url': 'https://www.amazon.com/Where-Sidewalk-Ends-Silverstein-Anniversary/dp/0060291699', 'count': '1,074,984', 'average': '4.29', '5_star': 53, '4_star': 28, '3_star': 13, '2_star': 2, '1_star': 1, 'goodreads_id': '30119'}
99 99 {'title': 'Where the Wild Things Are', 'url': 'https://www.amazon.com/Where-Wild-Things-Maurice-Sendak/dp/0060254920', 'count': '697,226', 'average': '4.22', '5_star': 52, '4_star': 25, '3_star': 14, '2_star': 4, '1_star': 2, 'goodreads_id': '19543'}


In [57]:
goodreads_books

[{'title': '1984',
  'url': 'https://www.amazon.com/1984-Signet-Classics-George-Orwell/dp/0451524934',
  'count': '2,352,493',
  'average': '4.16',
  '5_star': 45,
  '4_star': 33,
  '3_star': 15,
  '2_star': 4,
  '1_star': 2,
  'goodreads_id': '40961427'},
 {'title': 'A Brief History of Time',
  'url': 'https://www.amazon.com/Brief-History-Time-Stephen-Hawking/dp/0553380168',
  'count': '212,389',
  'average': '4.15',
  '5_star': 41,
  '4_star': 38,
  '3_star': 16,
  '2_star': 3,
  '1_star': 1,
  'goodreads_id': '3869'},
 {'title': 'A Heartbreaking Work of Staggering Genius',
  'url': 'https://www.amazon.com/Heartbreaking-Work-Staggering-Genius/dp/0375725784',
  'count': '156,244',
  'average': '3.68',
  '5_star': 27,
  '4_star': 33,
  '3_star': 23,
  '2_star': 10,
  '1_star': 5,
  'goodreads_id': '4953'},
 {'title': 'A Long Way Gone: Memoirs of a Boy Soldier',
  'url': 'https://www.amazon.com/Long-Way-Gone-Memoirs-Soldier/dp/0374531269',
  'count': '138,691',
  'average': '4.15',
  '5

In [58]:
goodreads_df = pd.DataFrame(goodreads_books)
goodreads_df.head()

,1_star,2_star,3_star,4_star,5_star,average,count,goodreads_id,title,url
0,2,4,15,33,45,4.16,"2,352,493",40961427,1984,https://www.amazon.com/1984-Signet-Classics-Ge...
1,1,3,16,38,41,4.15,"212,389",3869,A Brief History of Time,https://www.amazon.com/Brief-History-Time-Step...
2,5,10,23,33,27,3.68,"156,244",4953,A Heartbreaking Work of Staggering Genius,https://www.amazon.com/Heartbreaking-Work-Stag...
3,1,3,16,37,41,4.15,"138,691",43015,A Long Way Gone: Memoirs of a Boy Soldier,https://www.amazon.com/Long-Way-Gone-Memoirs-S...
4,2,6,23,34,33,3.91,"322,440",78411,"The Bad Beginning: Or, Orphans!",https://www.amazon.com/Bad-Beginning-Orphans-U...


In [60]:
goodreads_df.to_csv("goodreads.csv", header=None)